# Modelling decisions from experience

In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Embedding, Conv1D, MaxPooling1D
from keras import optimizers,regularizers

/m/home/home0/04/khandas1/data/Desktop/mtp/decisions-from-experience/env/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


### Gathering the dataset

In [2]:
dataset = pd.read_csv('data/estimation.csv', header=None)
dataset_comp = pd.read_csv('data/competition.csv', header=None)

In [3]:
X_train = dataset.iloc[:, 0:-1].values
y_train = dataset.iloc[:, -1].values
X_test = dataset_comp.iloc[:, 0:-1].values
y_test = dataset_comp.iloc[:, -1].values

In [4]:
input_length = X_train.shape[1]

### Preprocessing

In [5]:
# for efficient log-loss computation
y_train -= 1  
y_test -= 1  

In [6]:
# to feed the dataset as timesteps required for the LSTM training
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]//input_length, input_length))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]//input_length, input_length))

### Basic sequential model

In [7]:
model = Sequential()
model.add(LSTM(100, input_shape=(None, input_length)))
model.add(Dense(1, activation='sigmoid'))

In [8]:
opt = optimizers.Adam()
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               299600    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 299,701
Trainable params: 299,701
Non-trainable params: 0
_________________________________________________________________


### Fit the model

In [10]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=20)

Train on 1170 samples, validate on 1198 samples
Epoch 1/100
1170/1170 [==============================] - 2s 1ms/step - loss: 0.6944 - acc: 0.5718 - val_loss: 0.7018 - val_acc: 0.5217
Epoch 2/100
1170/1170 [==============================] - 1s 599us/step - loss: 0.6598 - acc: 0.6051 - val_loss: 0.6840 - val_acc: 0.5467
Epoch 3/100
1170/1170 [==============================] - 1s 594us/step - loss: 0.6465 - acc: 0.6282 - val_loss: 0.6798 - val_acc: 0.5668
Epoch 4/100
1170/1170 [==============================] - 1s 583us/step - loss: 0.6309 - acc: 0.6513 - val_loss: 0.6981 - val_acc: 0.5342
Epoch 5/100
1170/1170 [==============================] - 1s 601us/step - loss: 0.6194 - acc: 0.6658 - val_loss: 0.7117 - val_acc: 0.5509
Epoch 6/100
1170/1170 [==============================] - 1s 641us/step - loss: 0.6049 - acc: 0.6846 - val_loss: 0.7121 - val_acc: 0.5501
Epoch 7/100
1170/1170 [==============================] - 1s 667us/step - loss: 0.6004 - acc: 0.6744 - val_loss: 0.6989 - val_acc: 0.

1170/1170 [==============================] - 1s 565us/step - loss: 0.1883 - acc: 0.9291 - val_loss: 1.1714 - val_acc: 0.6152
Epoch 61/100
1170/1170 [==============================] - 1s 601us/step - loss: 0.1866 - acc: 0.9299 - val_loss: 1.1692 - val_acc: 0.6169
Epoch 62/100
1170/1170 [==============================] - 1s 547us/step - loss: 0.1803 - acc: 0.9350 - val_loss: 1.1740 - val_acc: 0.6135
Epoch 63/100
1170/1170 [==============================] - 1s 548us/step - loss: 0.1817 - acc: 0.9342 - val_loss: 1.2303 - val_acc: 0.6077
Epoch 64/100
1170/1170 [==============================] - 1s 549us/step - loss: 0.1773 - acc: 0.9368 - val_loss: 1.2008 - val_acc: 0.6119
Epoch 65/100
1170/1170 [==============================] - 1s 561us/step - loss: 0.1749 - acc: 0.9350 - val_loss: 1.2054 - val_acc: 0.6102
Epoch 66/100
1170/1170 [==============================] - 1s 547us/step - loss: 0.1809 - acc: 0.9299 - val_loss: 1.2278 - val_acc: 0.6002
Epoch 67/100
1170/1170 [=======================

In [11]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 61.52%


### Sensitivity Analysis

#### No. of epochs

In [12]:
outfile = open('results/diff-no-epochs.txt', 'w')
no_epochs = [1,3,5,10,30,50,100,150,200,300]

for epochs in no_epochs:
    model = Sequential()
    model.add(LSTM(100, input_shape=(None, input_length)))
    model.add(Dense(1, activation='sigmoid'))

    opt = optimizers.Adam()
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=20)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    scores1 = model.evaluate(X_train, y_train, verbose=0)
    print("Accuracy: %.2f%%" % (scores1[1]*100))
    outfile.write(str((epochs, scores[1]*100, scores1[1]*100)))
outfile.close()

Train on 1170 samples, validate on 1198 samples
Epoch 1/1
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6873 - acc: 0.5692 - val_loss: 0.6851 - val_acc: 0.5534
Accuracy: 55.34%
Accuracy: 62.22%
Train on 1170 samples, validate on 1198 samples
Epoch 1/3
1170/1170 [==============================] - 2s 1ms/step - loss: 0.6865 - acc: 0.5838 - val_loss: 0.6829 - val_acc: 0.5609
Epoch 2/3
1170/1170 [==============================] - 1s 523us/step - loss: 0.6575 - acc: 0.6231 - val_loss: 0.7013 - val_acc: 0.5359
Epoch 3/3
1170/1170 [==============================] - 1s 431us/step - loss: 0.6412 - acc: 0.6359 - val_loss: 0.6862 - val_acc: 0.5609
Accuracy: 56.09%
Accuracy: 66.07%
Train on 1170 samples, validate on 1198 samples
Epoch 1/5
1170/1170 [==============================] - 2s 2ms/step - loss: 0.6841 - acc: 0.5615 - val_loss: 0.6940 - val_acc: 0.5376
Epoch 2/5
1170/1170 [==============================] - 1s 515us/step - loss: 0.6521 - acc: 0.6128 - val_loss: 0.6787 - 

Epoch 9/50
1170/1170 [==============================] - 1s 597us/step - loss: 0.5754 - acc: 0.7060 - val_loss: 0.7045 - val_acc: 0.5927
Epoch 10/50
1170/1170 [==============================] - 1s 666us/step - loss: 0.5598 - acc: 0.7017 - val_loss: 0.7313 - val_acc: 0.5710
Epoch 11/50
1170/1170 [==============================] - 1s 657us/step - loss: 0.5496 - acc: 0.7154 - val_loss: 0.7349 - val_acc: 0.5843
Epoch 12/50
1170/1170 [==============================] - 1s 661us/step - loss: 0.5370 - acc: 0.7368 - val_loss: 0.7435 - val_acc: 0.5876
Epoch 13/50
1170/1170 [==============================] - 1s 629us/step - loss: 0.5210 - acc: 0.7504 - val_loss: 0.7350 - val_acc: 0.5743
Epoch 14/50
1170/1170 [==============================] - 1s 742us/step - loss: 0.5139 - acc: 0.7453 - val_loss: 0.7684 - val_acc: 0.5851
Epoch 15/50
1170/1170 [==============================] - 1s 659us/step - loss: 0.4939 - acc: 0.7675 - val_loss: 0.7618 - val_acc: 0.5826
Epoch 16/50
1170/1170 [===================

1170/1170 [==============================] - 1s 683us/step - loss: 0.4458 - acc: 0.7863 - val_loss: 0.8020 - val_acc: 0.5818
Epoch 19/100
1170/1170 [==============================] - 1s 480us/step - loss: 0.4328 - acc: 0.8094 - val_loss: 0.8476 - val_acc: 0.5718
Epoch 20/100
1170/1170 [==============================] - 0s 406us/step - loss: 0.4303 - acc: 0.7974 - val_loss: 0.8266 - val_acc: 0.5751
Epoch 21/100
1170/1170 [==============================] - 0s 404us/step - loss: 0.4098 - acc: 0.8248 - val_loss: 0.8413 - val_acc: 0.6043
Epoch 22/100
1170/1170 [==============================] - 0s 408us/step - loss: 0.3961 - acc: 0.8231 - val_loss: 0.8541 - val_acc: 0.6027
Epoch 23/100
1170/1170 [==============================] - 0s 389us/step - loss: 0.3759 - acc: 0.8444 - val_loss: 0.8709 - val_acc: 0.6002
Epoch 24/100
1170/1170 [==============================] - 0s 373us/step - loss: 0.3701 - acc: 0.8470 - val_loss: 0.8939 - val_acc: 0.6002
Epoch 25/100
1170/1170 [=======================

1170/1170 [==============================] - 1s 432us/step - loss: 0.1565 - acc: 0.9453 - val_loss: 1.3510 - val_acc: 0.6219
Epoch 78/100
1170/1170 [==============================] - 1s 429us/step - loss: 0.1518 - acc: 0.9427 - val_loss: 1.3596 - val_acc: 0.6068
Epoch 79/100
1170/1170 [==============================] - 0s 416us/step - loss: 0.1523 - acc: 0.9410 - val_loss: 1.3685 - val_acc: 0.6119
Epoch 80/100
1170/1170 [==============================] - 1s 489us/step - loss: 0.1504 - acc: 0.9462 - val_loss: 1.3616 - val_acc: 0.6210
Epoch 81/100
1170/1170 [==============================] - 0s 412us/step - loss: 0.1490 - acc: 0.9453 - val_loss: 1.4271 - val_acc: 0.6219
Epoch 82/100
1170/1170 [==============================] - 1s 483us/step - loss: 0.1490 - acc: 0.9487 - val_loss: 1.4026 - val_acc: 0.6144
Epoch 83/100
1170/1170 [==============================] - 1s 563us/step - loss: 0.1473 - acc: 0.9487 - val_loss: 1.3818 - val_acc: 0.6269
Epoch 84/100
1170/1170 [=======================

Epoch 36/150
1170/1170 [==============================] - 1s 564us/step - loss: 0.2879 - acc: 0.8846 - val_loss: 0.9884 - val_acc: 0.6043
Epoch 37/150
1170/1170 [==============================] - 1s 598us/step - loss: 0.2746 - acc: 0.8932 - val_loss: 1.0175 - val_acc: 0.5935
Epoch 38/150
1170/1170 [==============================] - 1s 506us/step - loss: 0.2729 - acc: 0.8949 - val_loss: 1.0322 - val_acc: 0.5977
Epoch 39/150
1170/1170 [==============================] - 1s 478us/step - loss: 0.2622 - acc: 0.8991 - val_loss: 1.0269 - val_acc: 0.6102
Epoch 40/150
1170/1170 [==============================] - 1s 453us/step - loss: 0.2583 - acc: 0.8991 - val_loss: 1.0535 - val_acc: 0.6119
Epoch 41/150
1170/1170 [==============================] - 1s 482us/step - loss: 0.2541 - acc: 0.9017 - val_loss: 1.0427 - val_acc: 0.6052
Epoch 42/150
1170/1170 [==============================] - 0s 403us/step - loss: 0.2538 - acc: 0.9017 - val_loss: 1.0603 - val_acc: 0.6035
Epoch 43/150
1170/1170 [==========

...

Epoch 39/300
1170/1170 [==============================] - 1s 450us/step - loss: 0.2721 - acc: 0.8949 - val_loss: 1.0240 - val_acc: 0.6127
Epoch 40/300
1170/1170 [==============================] - 1s 611us/step - loss: 0.2620 - acc: 0.9000 - val_loss: 1.0331 - val_acc: 0.6110
Epoch 41/300
1170/1170 [==============================] - 1s 551us/step - loss: 0.2598 - acc: 0.9009 - val_loss: 1.0358 - val_acc: 0.6060
Epoch 42/300
1170/1170 [==============================] - 1s 486us/step - loss: 0.2567 - acc: 0.9034 - val_loss: 1.0338 - val_acc: 0.6202
Epoch 43/300
1170/1170 [==============================] - 1s 512us/step - loss: 0.2473 - acc: 0.9043 - val_loss: 1.0453 - val_acc: 0.6194
Epoch 44/300
1170/1170 [==============================] - 1s 579us/step - loss: 0.2482 - acc: 0.9000 - val_loss: 1.0814 - val_acc: 0.6160
Epoch 45/300
1170/1170 [==============================] - 1s 614us/step - loss: 0.2408 - acc: 0.9085 - val_loss: 1.0537 - val_acc: 0.6202
Epoch 46/300
1170/1170 [==========

1170/1170 [==============================] - 1s 638us/step - loss: 0.1291 - acc: 0.9556 - val_loss: 1.5090 - val_acc: 0.6194
Epoch 99/300
1170/1170 [==============================] - 1s 758us/step - loss: 0.1273 - acc: 0.9573 - val_loss: 1.4748 - val_acc: 0.6152
Epoch 100/300
1170/1170 [==============================] - 1s 674us/step - loss: 0.1297 - acc: 0.9573 - val_loss: 1.4756 - val_acc: 0.6052
Epoch 101/300
1170/1170 [==============================] - 1s 680us/step - loss: 0.1282 - acc: 0.9530 - val_loss: 1.4803 - val_acc: 0.6144
Epoch 102/300
1170/1170 [==============================] - 1s 643us/step - loss: 0.1287 - acc: 0.9521 - val_loss: 1.4959 - val_acc: 0.6102
Epoch 103/300
1170/1170 [==============================] - 1s 718us/step - loss: 0.1252 - acc: 0.9521 - val_loss: 1.4942 - val_acc: 0.6110
Epoch 104/300
1170/1170 [==============================] - 1s 661us/step - loss: 0.1295 - acc: 0.9513 - val_loss: 1.5014 - val_acc: 0.6169
Epoch 105/300
1170/1170 [=================

1170/1170 [==============================] - 1s 565us/step - loss: 0.1034 - acc: 0.9615 - val_loss: 1.6348 - val_acc: 0.5985
Epoch 158/300
1170/1170 [==============================] - 1s 597us/step - loss: 0.1005 - acc: 0.9615 - val_loss: 1.6174 - val_acc: 0.5935
Epoch 159/300
1170/1170 [==============================] - 1s 589us/step - loss: 0.0974 - acc: 0.9658 - val_loss: 1.6473 - val_acc: 0.5985
Epoch 160/300
1170/1170 [==============================] - 1s 569us/step - loss: 0.0979 - acc: 0.9658 - val_loss: 1.6580 - val_acc: 0.5943
Epoch 161/300
1170/1170 [==============================] - 1s 583us/step - loss: 0.0988 - acc: 0.9650 - val_loss: 1.6675 - val_acc: 0.5977
Epoch 162/300
1170/1170 [==============================] - 1s 568us/step - loss: 0.0977 - acc: 0.9667 - val_loss: 1.6556 - val_acc: 0.5993
Epoch 163/300
1170/1170 [==============================] - 1s 669us/step - loss: 0.1023 - acc: 0.9598 - val_loss: 1.6788 - val_acc: 0.5985
Epoch 164/300
1170/1170 [================

1170/1170 [==============================] - 0s 382us/step - loss: 0.0879 - acc: 0.9675 - val_loss: 1.6757 - val_acc: 0.5952
Epoch 217/300
1170/1170 [==============================] - 0s 415us/step - loss: 0.0872 - acc: 0.9641 - val_loss: 1.6759 - val_acc: 0.5968
Epoch 218/300
1170/1170 [==============================] - 0s 395us/step - loss: 0.0890 - acc: 0.9641 - val_loss: 1.7042 - val_acc: 0.6002
Epoch 219/300
1170/1170 [==============================] - 0s 425us/step - loss: 0.0891 - acc: 0.9684 - val_loss: 1.6888 - val_acc: 0.6018
Epoch 220/300
1170/1170 [==============================] - 0s 398us/step - loss: 0.0927 - acc: 0.9667 - val_loss: 1.6903 - val_acc: 0.6068
Epoch 221/300
1170/1170 [==============================] - 0s 397us/step - loss: 0.0865 - acc: 0.9718 - val_loss: 1.7198 - val_acc: 0.6010
Epoch 222/300
1170/1170 [==============================] - 0s 383us/step - loss: 0.0918 - acc: 0.9650 - val_loss: 1.7181 - val_acc: 0.6027
Epoch 223/300
1170/1170 [================

1170/1170 [==============================] - 1s 547us/step - loss: 0.0784 - acc: 0.9701 - val_loss: 1.7937 - val_acc: 0.5968
Epoch 276/300
1170/1170 [==============================] - 1s 590us/step - loss: 0.0805 - acc: 0.9658 - val_loss: 1.8097 - val_acc: 0.6052
Epoch 277/300
1170/1170 [==============================] - 1s 628us/step - loss: 0.0787 - acc: 0.9684 - val_loss: 1.8224 - val_acc: 0.6027
Epoch 278/300
1170/1170 [==============================] - 1s 627us/step - loss: 0.0778 - acc: 0.9718 - val_loss: 1.8067 - val_acc: 0.6052
Epoch 279/300
1170/1170 [==============================] - 1s 698us/step - loss: 0.0800 - acc: 0.9684 - val_loss: 1.8159 - val_acc: 0.5985
Epoch 280/300
1170/1170 [==============================] - 1s 615us/step - loss: 0.0827 - acc: 0.9658 - val_loss: 1.8235 - val_acc: 0.6035
Epoch 281/300
1170/1170 [==============================] - 1s 662us/step - loss: 0.0791 - acc: 0.9667 - val_loss: 1.8405 - val_acc: 0.6068
Epoch 282/300
1170/1170 [================

#### Batch size

In [13]:
outfile = open('results/diff-no-bs.txt', 'w')
no_bs = [1,3,5,10,30,50,100]

for bs in no_bs:
    model = Sequential()
    model.add(LSTM(100, input_shape=(None, input_length)))
    model.add(Dense(1, activation='sigmoid'))

    opt = optimizers.Adam()
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=bs)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    scores1 = model.evaluate(X_train, y_train, verbose=0)
    print("Accuracy: %.2f%%" % (scores1[1]*100))
    outfile.write(str((bs, scores[1]*100, scores1[1]*100)))
outfile.close()

Train on 1170 samples, validate on 1198 samples
Epoch 1/50
1170/1170 [==============================] - 16s 13ms/step - loss: 0.6941 - acc: 0.5812 - val_loss: 0.6957 - val_acc: 0.5442
Epoch 2/50
1170/1170 [==============================] - 10s 9ms/step - loss: 0.6666 - acc: 0.5940 - val_loss: 0.6972 - val_acc: 0.5225
Epoch 3/50
1170/1170 [==============================] - 13s 11ms/step - loss: 0.6594 - acc: 0.5949 - val_loss: 0.7120 - val_acc: 0.5292
Epoch 4/50
1170/1170 [==============================] - 12s 10ms/step - loss: 0.6399 - acc: 0.6248 - val_loss: 0.7022 - val_acc: 0.5384
Epoch 5/50
1170/1170 [==============================] - 12s 11ms/step - loss: 0.6290 - acc: 0.6427 - val_loss: 0.6931 - val_acc: 0.5743
Epoch 6/50
1170/1170 [==============================] - 12s 10ms/step - loss: 0.6068 - acc: 0.6650 - val_loss: 0.6849 - val_acc: 0.5860
Epoch 7/50
1170/1170 [==============================] - 13s 11ms/step - loss: 0.5898 - acc: 0.6821 - val_loss: 0.7052 - val_acc: 0.5743
E

1170/1170 [==============================] - 5s 4ms/step - loss: 0.5292 - acc: 0.7410 - val_loss: 0.7376 - val_acc: 0.5776
Epoch 11/50
1170/1170 [==============================] - 5s 4ms/step - loss: 0.5152 - acc: 0.7359 - val_loss: 0.7654 - val_acc: 0.5659
Epoch 12/50
1170/1170 [==============================] - 5s 4ms/step - loss: 0.4930 - acc: 0.7581 - val_loss: 0.7631 - val_acc: 0.5860
Epoch 13/50
1170/1170 [==============================] - 4s 4ms/step - loss: 0.4768 - acc: 0.7769 - val_loss: 0.7561 - val_acc: 0.5952
Epoch 14/50
1170/1170 [==============================] - 4s 4ms/step - loss: 0.4577 - acc: 0.7855 - val_loss: 0.8114 - val_acc: 0.5743
Epoch 15/50
1170/1170 [==============================] - 4s 3ms/step - loss: 0.4447 - acc: 0.8017 - val_loss: 0.7945 - val_acc: 0.5751
Epoch 16/50
1170/1170 [==============================] - 5s 4ms/step - loss: 0.4232 - acc: 0.8085 - val_loss: 0.8254 - val_acc: 0.5818
Epoch 17/50
1170/1170 [==============================] - 5s 4ms/ste

1170/1170 [==============================] - 2s 2ms/step - loss: 0.3778 - acc: 0.8385 - val_loss: 0.8570 - val_acc: 0.6052
Epoch 21/50
1170/1170 [==============================] - 3s 2ms/step - loss: 0.3685 - acc: 0.8462 - val_loss: 0.8556 - val_acc: 0.5993
Epoch 22/50
1170/1170 [==============================] - 3s 3ms/step - loss: 0.3493 - acc: 0.8632 - val_loss: 0.8782 - val_acc: 0.6027
Epoch 23/50
1170/1170 [==============================] - 3s 2ms/step - loss: 0.3405 - acc: 0.8487 - val_loss: 0.8986 - val_acc: 0.5893
Epoch 24/50
1170/1170 [==============================] - 3s 2ms/step - loss: 0.3250 - acc: 0.8726 - val_loss: 0.9108 - val_acc: 0.6185
Epoch 25/50
1170/1170 [==============================] - 3s 2ms/step - loss: 0.3185 - acc: 0.8769 - val_loss: 0.9191 - val_acc: 0.6027
Epoch 26/50
1170/1170 [==============================] - 3s 2ms/step - loss: 0.2939 - acc: 0.8915 - val_loss: 0.9278 - val_acc: 0.6043
Epoch 27/50
1170/1170 [==============================] - 3s 2ms/ste

1170/1170 [==============================] - 1s 847us/step - loss: 0.2858 - acc: 0.8923 - val_loss: 0.9807 - val_acc: 0.6194
Epoch 31/50
1170/1170 [==============================] - 1s 851us/step - loss: 0.2761 - acc: 0.8880 - val_loss: 1.0200 - val_acc: 0.6093
Epoch 32/50
1170/1170 [==============================] - 1s 936us/step - loss: 0.2777 - acc: 0.8974 - val_loss: 0.9888 - val_acc: 0.6177
Epoch 33/50
1170/1170 [==============================] - 1s 778us/step - loss: 0.2666 - acc: 0.8991 - val_loss: 1.0264 - val_acc: 0.6010
Epoch 34/50
1170/1170 [==============================] - 1s 825us/step - loss: 0.2528 - acc: 0.9009 - val_loss: 1.0281 - val_acc: 0.6152
Epoch 35/50
1170/1170 [==============================] - 1s 788us/step - loss: 0.2486 - acc: 0.9026 - val_loss: 1.0451 - val_acc: 0.6102
Epoch 36/50
1170/1170 [==============================] - 1s 846us/step - loss: 0.2406 - acc: 0.9145 - val_loss: 1.0893 - val_acc: 0.6119
Epoch 37/50
1170/1170 [==============================

1170/1170 [==============================] - 1s 430us/step - loss: 0.2905 - acc: 0.8906 - val_loss: 0.9756 - val_acc: 0.6052
Epoch 40/50
1170/1170 [==============================] - 1s 464us/step - loss: 0.2914 - acc: 0.8872 - val_loss: 0.9777 - val_acc: 0.6060
Epoch 41/50
1170/1170 [==============================] - 0s 405us/step - loss: 0.2803 - acc: 0.8906 - val_loss: 0.9913 - val_acc: 0.6144
Epoch 42/50
1170/1170 [==============================] - 1s 454us/step - loss: 0.2698 - acc: 0.8974 - val_loss: 1.0170 - val_acc: 0.6194
Epoch 43/50
1170/1170 [==============================] - 0s 388us/step - loss: 0.2649 - acc: 0.9000 - val_loss: 1.0354 - val_acc: 0.6227
Epoch 44/50
1170/1170 [==============================] - 0s 400us/step - loss: 0.2627 - acc: 0.8957 - val_loss: 1.0265 - val_acc: 0.6152
Epoch 45/50
1170/1170 [==============================] - 0s 322us/step - loss: 0.2573 - acc: 0.9009 - val_loss: 1.0432 - val_acc: 0.6152
Epoch 46/50
1170/1170 [==============================

1170/1170 [==============================] - 0s 304us/step - loss: 0.2945 - acc: 0.8855 - val_loss: 0.9532 - val_acc: 0.6077
Epoch 49/50
1170/1170 [==============================] - 0s 285us/step - loss: 0.2913 - acc: 0.8906 - val_loss: 0.9608 - val_acc: 0.6202
Epoch 50/50
1170/1170 [==============================] - 0s 284us/step - loss: 0.2804 - acc: 0.8923 - val_loss: 0.9788 - val_acc: 0.6077
Accuracy: 60.77%
Accuracy: 90.77%
Train on 1170 samples, validate on 1198 samples
Epoch 1/50
1170/1170 [==============================] - 3s 3ms/step - loss: 0.6797 - acc: 0.5846 - val_loss: 0.6832 - val_acc: 0.5484
Epoch 2/50
1170/1170 [==============================] - 0s 157us/step - loss: 0.6554 - acc: 0.6111 - val_loss: 0.6902 - val_acc: 0.5317
Epoch 3/50
1170/1170 [==============================] - 0s 194us/step - loss: 0.6430 - acc: 0.6368 - val_loss: 0.6880 - val_acc: 0.5459
Epoch 4/50
1170/1170 [==============================] - 0s 141us/step - loss: 0.6327 - acc: 0.6444 - val_loss: 0.

#### No. of LSTM units

In [14]:
outfile = open('results/diff-no-units.txt', 'w')
no_units = [1,3,5,10,30,50,100,300,500]

for units in no_units:
    model = Sequential()
    model.add(LSTM(units, input_shape=(None, input_length)))
    model.add(Dense(1, activation='sigmoid'))

    opt = optimizers.Adam()
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=20)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    scores1 = model.evaluate(X_train, y_train, verbose=0)
    print("Accuracy: %.2f%%" % (scores1[1]*100))
    outfile.write(str((units, scores[1]*100, scores1[1]*100)))
outfile.close()

Train on 1170 samples, validate on 1198 samples
Epoch 1/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.6886 - acc: 0.5701 - val_loss: 0.6927 - val_acc: 0.5217
Epoch 2/100
1170/1170 [==============================] - 0s 209us/step - loss: 0.6863 - acc: 0.5957 - val_loss: 0.6933 - val_acc: 0.5259
Epoch 3/100
1170/1170 [==============================] - 0s 235us/step - loss: 0.6844 - acc: 0.5923 - val_loss: 0.6930 - val_acc: 0.5317
Epoch 4/100
1170/1170 [==============================] - 0s 270us/step - loss: 0.6824 - acc: 0.5983 - val_loss: 0.6910 - val_acc: 0.5284
Epoch 5/100
1170/1170 [==============================] - 0s 326us/step - loss: 0.6804 - acc: 0.6051 - val_loss: 0.6901 - val_acc: 0.5359
Epoch 6/100
1170/1170 [==============================] - 0s 264us/step - loss: 0.6785 - acc: 0.5974 - val_loss: 0.6915 - val_acc: 0.5384
Epoch 7/100
1170/1170 [==============================] - 0s 222us/step - loss: 0.6769 - acc: 0.5983 - val_loss: 0.6910 - val_acc: 0.

1170/1170 [==============================] - 0s 367us/step - loss: 0.6072 - acc: 0.6650 - val_loss: 0.7103 - val_acc: 0.5668
Epoch 61/100
1170/1170 [==============================] - 0s 362us/step - loss: 0.6075 - acc: 0.6556 - val_loss: 0.7054 - val_acc: 0.5735
Epoch 62/100
1170/1170 [==============================] - 0s 399us/step - loss: 0.6061 - acc: 0.6581 - val_loss: 0.7016 - val_acc: 0.5776
Epoch 63/100
1170/1170 [==============================] - 0s 387us/step - loss: 0.6066 - acc: 0.6632 - val_loss: 0.7090 - val_acc: 0.5726
Epoch 64/100
1170/1170 [==============================] - 0s 372us/step - loss: 0.6051 - acc: 0.6598 - val_loss: 0.7034 - val_acc: 0.5751
Epoch 65/100
1170/1170 [==============================] - 0s 302us/step - loss: 0.6029 - acc: 0.6658 - val_loss: 0.7099 - val_acc: 0.5735
Epoch 66/100
1170/1170 [==============================] - 0s 325us/step - loss: 0.6040 - acc: 0.6607 - val_loss: 0.7018 - val_acc: 0.5818
Epoch 67/100
1170/1170 [=======================

Epoch 19/100
1170/1170 [==============================] - 0s 329us/step - loss: 0.6230 - acc: 0.6650 - val_loss: 0.6935 - val_acc: 0.5401
Epoch 20/100
1170/1170 [==============================] - 0s 328us/step - loss: 0.6205 - acc: 0.6684 - val_loss: 0.6926 - val_acc: 0.5501
Epoch 21/100
1170/1170 [==============================] - 0s 277us/step - loss: 0.6161 - acc: 0.6692 - val_loss: 0.6973 - val_acc: 0.5342
Epoch 22/100
1170/1170 [==============================] - 0s 298us/step - loss: 0.6137 - acc: 0.6709 - val_loss: 0.6964 - val_acc: 0.5476
Epoch 23/100
1170/1170 [==============================] - 0s 302us/step - loss: 0.6105 - acc: 0.6735 - val_loss: 0.6951 - val_acc: 0.5543
Epoch 24/100
1170/1170 [==============================] - 0s 354us/step - loss: 0.6067 - acc: 0.6795 - val_loss: 0.6984 - val_acc: 0.5442
Epoch 25/100
1170/1170 [==============================] - 0s 248us/step - loss: 0.6060 - acc: 0.6812 - val_loss: 0.7008 - val_acc: 0.5442
Epoch 26/100
1170/1170 [==========

1170/1170 [==============================] - 0s 321us/step - loss: 0.4864 - acc: 0.7684 - val_loss: 0.7789 - val_acc: 0.5751
Epoch 79/100
1170/1170 [==============================] - 0s 390us/step - loss: 0.4829 - acc: 0.7632 - val_loss: 0.7766 - val_acc: 0.5768
Epoch 80/100
1170/1170 [==============================] - 0s 320us/step - loss: 0.4857 - acc: 0.7632 - val_loss: 0.7840 - val_acc: 0.5818
Epoch 81/100
1170/1170 [==============================] - 0s 383us/step - loss: 0.4793 - acc: 0.7607 - val_loss: 0.7788 - val_acc: 0.5718
Epoch 82/100
1170/1170 [==============================] - 0s 328us/step - loss: 0.4813 - acc: 0.7624 - val_loss: 0.7850 - val_acc: 0.5876
Epoch 83/100
1170/1170 [==============================] - 0s 232us/step - loss: 0.4774 - acc: 0.7735 - val_loss: 0.7903 - val_acc: 0.5801
Epoch 84/100
1170/1170 [==============================] - 0s 277us/step - loss: 0.4750 - acc: 0.7769 - val_loss: 0.7925 - val_acc: 0.5785
Epoch 85/100
1170/1170 [=======================

Epoch 37/100
1170/1170 [==============================] - 0s 364us/step - loss: 0.5119 - acc: 0.7547 - val_loss: 0.7418 - val_acc: 0.5743
Epoch 38/100
1170/1170 [==============================] - 0s 313us/step - loss: 0.5085 - acc: 0.7624 - val_loss: 0.7462 - val_acc: 0.5776
Epoch 39/100
1170/1170 [==============================] - 0s 385us/step - loss: 0.5111 - acc: 0.7521 - val_loss: 0.7536 - val_acc: 0.5676
Epoch 40/100
1170/1170 [==============================] - 0s 282us/step - loss: 0.5031 - acc: 0.7692 - val_loss: 0.7421 - val_acc: 0.5634
Epoch 41/100
1170/1170 [==============================] - 0s 255us/step - loss: 0.4975 - acc: 0.7709 - val_loss: 0.7536 - val_acc: 0.5718
Epoch 42/100
1170/1170 [==============================] - 0s 243us/step - loss: 0.4955 - acc: 0.7718 - val_loss: 0.7558 - val_acc: 0.5743
Epoch 43/100
1170/1170 [==============================] - 0s 294us/step - loss: 0.4923 - acc: 0.7684 - val_loss: 0.7682 - val_acc: 0.5701
Epoch 44/100
1170/1170 [==========

...

Epoch 49/100
1170/1170 [==============================] - 1s 587us/step - loss: 0.2176 - acc: 0.9222 - val_loss: 1.1130 - val_acc: 0.6035
Epoch 50/100
1170/1170 [==============================] - 1s 585us/step - loss: 0.2134 - acc: 0.9214 - val_loss: 1.0847 - val_acc: 0.6185
Epoch 51/100
1170/1170 [==============================] - 1s 628us/step - loss: 0.2070 - acc: 0.9265 - val_loss: 1.1136 - val_acc: 0.6169
Epoch 52/100
1170/1170 [==============================] - 1s 609us/step - loss: 0.2144 - acc: 0.9197 - val_loss: 1.1353 - val_acc: 0.6018
Epoch 53/100
1170/1170 [==============================] - 1s 624us/step - loss: 0.2092 - acc: 0.9145 - val_loss: 1.1340 - val_acc: 0.6127
Epoch 54/100
1170/1170 [==============================] - 1s 540us/step - loss: 0.2043 - acc: 0.9162 - val_loss: 1.1397 - val_acc: 0.6152
Epoch 55/100
1170/1170 [==============================] - 1s 581us/step - loss: 0.1981 - acc: 0.9291 - val_loss: 1.1339 - val_acc: 0.6119
Epoch 56/100
1170/1170 [==========

Epoch 8/100
1170/1170 [==============================] - 2s 1ms/step - loss: 0.5667 - acc: 0.7068 - val_loss: 0.7417 - val_acc: 0.5868
Epoch 9/100
1170/1170 [==============================] - 2s 1ms/step - loss: 0.5533 - acc: 0.7179 - val_loss: 0.7518 - val_acc: 0.5751
Epoch 10/100
1170/1170 [==============================] - 2s 1ms/step - loss: 0.5357 - acc: 0.7333 - val_loss: 0.7874 - val_acc: 0.5634
Epoch 11/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.5296 - acc: 0.7350 - val_loss: 0.7784 - val_acc: 0.5818
Epoch 12/100
1170/1170 [==============================] - 2s 1ms/step - loss: 0.5056 - acc: 0.7667 - val_loss: 0.7856 - val_acc: 0.5785
Epoch 13/100
1170/1170 [==============================] - 2s 1ms/step - loss: 0.5006 - acc: 0.7607 - val_loss: 0.7851 - val_acc: 0.5743
Epoch 14/100
1170/1170 [==============================] - 2s 1ms/step - loss: 0.4821 - acc: 0.7726 - val_loss: 0.8120 - val_acc: 0.5684
Epoch 15/100
1170/1170 [==========================

1170/1170 [==============================] - 2s 2ms/step - loss: 0.1506 - acc: 0.9419 - val_loss: 1.4165 - val_acc: 0.6102
Epoch 69/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.1548 - acc: 0.9402 - val_loss: 1.4510 - val_acc: 0.6135
Epoch 70/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.1569 - acc: 0.9385 - val_loss: 1.4596 - val_acc: 0.6102
Epoch 71/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.1602 - acc: 0.9350 - val_loss: 1.4577 - val_acc: 0.6227
Epoch 72/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.1499 - acc: 0.9462 - val_loss: 1.4519 - val_acc: 0.6027
Epoch 73/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.1494 - acc: 0.9470 - val_loss: 1.5005 - val_acc: 0.6169
Epoch 74/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.1513 - acc: 0.9393 - val_loss: 1.5392 - val_acc: 0.6043
Epoch 75/100
1170/1170 [==============================] - 2s 

Epoch 28/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.3216 - acc: 0.8709 - val_loss: 1.0298 - val_acc: 0.5910
Epoch 29/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.3037 - acc: 0.8761 - val_loss: 1.0782 - val_acc: 0.5927
Epoch 30/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.2951 - acc: 0.8803 - val_loss: 1.0968 - val_acc: 0.6177
Epoch 31/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.2887 - acc: 0.8846 - val_loss: 1.1338 - val_acc: 0.6052
Epoch 32/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.2814 - acc: 0.8795 - val_loss: 1.0939 - val_acc: 0.5968
Epoch 33/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.2692 - acc: 0.8897 - val_loss: 1.1295 - val_acc: 0.6043
Epoch 34/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.2564 - acc: 0.9009 - val_loss: 1.2078 - val_acc: 0.6077
Epoch 35/100
1170/1170 [========================

1170/1170 [==============================] - 3s 3ms/step - loss: 0.1279 - acc: 0.9556 - val_loss: 1.6253 - val_acc: 0.6093
Epoch 89/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.1241 - acc: 0.9530 - val_loss: 1.6579 - val_acc: 0.6068
Epoch 90/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.1227 - acc: 0.9496 - val_loss: 1.6407 - val_acc: 0.6060
Epoch 91/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.1170 - acc: 0.9564 - val_loss: 1.6530 - val_acc: 0.6002
Epoch 92/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.1224 - acc: 0.9479 - val_loss: 1.6859 - val_acc: 0.6010
Epoch 93/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.1190 - acc: 0.9530 - val_loss: 1.6972 - val_acc: 0.5943
Epoch 94/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.1191 - acc: 0.9530 - val_loss: 1.7340 - val_acc: 0.6110
Epoch 95/100
1170/1170 [==============================] - 3s 

#### No. of layers

In [ ]:
outfile = open('results/diff-no-layers.txt', 'w')
no_layers = [1,2,3,5,7,10,20]

d = 0.2

for layers in no_layers:
    model = Sequential()
    if(layers == 1):
        model.add(LSTM(50, input_shape=(None, input_length)))
    else:
        model.add(LSTM(50, input_shape=(None, input_length), return_sequences=True))
        while(layers-2):
            model.add(LSTM(50, input_shape=(None, input_length), return_sequences=True))
            layers -= 1
        model.add(LSTM(50, input_shape=(None, input_length)))
    model.add(Dense(1, activation='sigmoid'))

    opt = optimizers.Adam()
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=20)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    scores1 = model.evaluate(X_train, y_train, verbose=0)
    print("Accuracy: %.2f%%" % (scores1[1]*100))
    outfile.write(str((layers, scores[1]*100, scores1[1]*100)))
outfile.close()

Train on 1170 samples, validate on 1198 samples
Epoch 1/50
1170/1170 [==============================] - 7s 6ms/step - loss: 0.6871 - acc: 0.5709 - val_loss: 0.6845 - val_acc: 0.5668
Epoch 2/50
1170/1170 [==============================] - 1s 456us/step - loss: 0.6655 - acc: 0.5949 - val_loss: 0.6823 - val_acc: 0.5634
Epoch 3/50
1170/1170 [==============================] - 1s 495us/step - loss: 0.6510 - acc: 0.6248 - val_loss: 0.6778 - val_acc: 0.5701
Epoch 4/50
1170/1170 [==============================] - 1s 500us/step - loss: 0.6426 - acc: 0.6419 - val_loss: 0.6841 - val_acc: 0.5626
Epoch 5/50
1170/1170 [==============================] - 1s 459us/step - loss: 0.6285 - acc: 0.6556 - val_loss: 0.6958 - val_acc: 0.5451
Epoch 6/50
1170/1170 [==============================] - 1s 467us/step - loss: 0.6180 - acc: 0.6667 - val_loss: 0.6877 - val_acc: 0.5634
Epoch 7/50
1170/1170 [==============================] - 1s 444us/step - loss: 0.6090 - acc: 0.6778 - val_loss: 0.6880 - val_acc: 0.5693
Ep

1170/1170 [==============================] - 1s 714us/step - loss: 0.5250 - acc: 0.7453 - val_loss: 0.7700 - val_acc: 0.5618
Epoch 11/50
1170/1170 [==============================] - 1s 595us/step - loss: 0.5191 - acc: 0.7547 - val_loss: 0.7934 - val_acc: 0.5776
Epoch 12/50
1170/1170 [==============================] - 1s 658us/step - loss: 0.4915 - acc: 0.7530 - val_loss: 0.8811 - val_acc: 0.5634
Epoch 13/50
1170/1170 [==============================] - 1s 629us/step - loss: 0.4771 - acc: 0.7803 - val_loss: 0.8602 - val_acc: 0.5785
Epoch 14/50
1170/1170 [==============================] - 1s 779us/step - loss: 0.4715 - acc: 0.7684 - val_loss: 0.9094 - val_acc: 0.5868
Epoch 15/50
1170/1170 [==============================] - 1s 709us/step - loss: 0.4349 - acc: 0.7880 - val_loss: 0.9491 - val_acc: 0.5818
Epoch 16/50
1170/1170 [==============================] - 1s 783us/step - loss: 0.4425 - acc: 0.7983 - val_loss: 0.9586 - val_acc: 0.5943
Epoch 17/50
1170/1170 [==============================

1170/1170 [==============================] - 1s 791us/step - loss: 0.3609 - acc: 0.8436 - val_loss: 1.1291 - val_acc: 0.5876
Epoch 20/50
1170/1170 [==============================] - 1s 804us/step - loss: 0.3384 - acc: 0.8444 - val_loss: 1.1849 - val_acc: 0.5927
Epoch 21/50
1170/1170 [==============================] - 1s 767us/step - loss: 0.3239 - acc: 0.8504 - val_loss: 1.2643 - val_acc: 0.5776
Epoch 22/50
1170/1170 [==============================] - 1s 759us/step - loss: 0.3451 - acc: 0.8470 - val_loss: 1.2359 - val_acc: 0.5835
Epoch 23/50
1170/1170 [==============================] - 1s 750us/step - loss: 0.3217 - acc: 0.8547 - val_loss: 1.2485 - val_acc: 0.5726
Epoch 24/50
1170/1170 [==============================] - 1s 784us/step - loss: 0.2940 - acc: 0.8718 - val_loss: 1.2883 - val_acc: 0.5910
Epoch 25/50
1170/1170 [==============================] - 1s 730us/step - loss: 0.2962 - acc: 0.8650 - val_loss: 1.3755 - val_acc: 0.5902
Epoch 26/50
1170/1170 [==============================

Epoch 29/50
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2546 - acc: 0.9034 - val_loss: 1.2502 - val_acc: 0.5885
Epoch 30/50
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2481 - acc: 0.9017 - val_loss: 1.2995 - val_acc: 0.5676
Epoch 31/50
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2664 - acc: 0.8906 - val_loss: 1.2947 - val_acc: 0.5718
Epoch 32/50
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2322 - acc: 0.9051 - val_loss: 1.4591 - val_acc: 0.5860
Epoch 33/50
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2728 - acc: 0.8940 - val_loss: 1.2918 - val_acc: 0.5860
Epoch 34/50
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2868 - acc: 0.8786 - val_loss: 1.2152 - val_acc: 0.5876
Epoch 35/50
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2526 - acc: 0.9026 - val_loss: 1.2996 - val_acc: 0.5843
Epoch 36/50
1170/1170 [==============================] 

1170/1170 [==============================] - 2s 2ms/step - loss: 0.2196 - acc: 0.9214 - val_loss: 1.2597 - val_acc: 0.5918
Epoch 40/50
1170/1170 [==============================] - 2s 1ms/step - loss: 0.2235 - acc: 0.9205 - val_loss: 1.2047 - val_acc: 0.6052
Epoch 41/50
1170/1170 [==============================] - 2s 1ms/step - loss: 0.2183 - acc: 0.9162 - val_loss: 1.2411 - val_acc: 0.5885
Epoch 42/50
1170/1170 [==============================] - 2s 2ms/step - loss: 0.2104 - acc: 0.9214 - val_loss: 1.2931 - val_acc: 0.6027
Epoch 43/50
1170/1170 [==============================] - 2s 2ms/step - loss: 0.2456 - acc: 0.9017 - val_loss: 1.2601 - val_acc: 0.6160
Epoch 44/50
1170/1170 [==============================] - 2s 2ms/step - loss: 0.2084 - acc: 0.9308 - val_loss: 1.2940 - val_acc: 0.6252
Epoch 45/50
1170/1170 [==============================] - 2s 2ms/step - loss: 0.2206 - acc: 0.9171 - val_loss: 1.2484 - val_acc: 0.5960
Epoch 46/50
1170/1170 [==============================] - 2s 1ms/ste

1170/1170 [==============================] - 3s 2ms/step - loss: 0.6931 - acc: 0.5111 - val_loss: 0.6908 - val_acc: 0.5543
Epoch 50/50
1170/1170 [==============================] - 2s 2ms/step - loss: 0.6931 - acc: 0.5111 - val_loss: 0.6915 - val_acc: 0.5543
Accuracy: 55.43%
Accuracy: 51.11%
Train on 1170 samples, validate on 1198 samples
Epoch 1/50
1170/1170 [==============================] - 33s 29ms/step - loss: 0.6932 - acc: 0.5026 - val_loss: 0.6921 - val_acc: 0.5543
Epoch 2/50
1170/1170 [==============================] - 4s 4ms/step - loss: 0.6931 - acc: 0.5111 - val_loss: 0.6918 - val_acc: 0.5543
Epoch 3/50
1170/1170 [==============================] - 5s 4ms/step - loss: 0.6931 - acc: 0.5111 - val_loss: 0.6915 - val_acc: 0.5543
Epoch 4/50
1170/1170 [==============================] - 4s 4ms/step - loss: 0.6930 - acc: 0.5111 - val_loss: 0.6912 - val_acc: 0.5543
Epoch 5/50
1170/1170 [==============================] - 5s 4ms/step - loss: 0.6931 - acc: 0.5111 - val_loss: 0.6912 - val_